# Files
See http://openbookproject.net/thinkcs/python/english3e/files.html

file_object = `open(`file_name`, mode='r')`

|Character	| Meaning           |
|:---------:|:------------------|
|`'r'`	| open for reading (default)|
|`'w'`	| open for writing, truncating the file first |
|`'x'`	| open for exclusive creation, failing if the file already exists |
|`'a'`	| open for writing, appending to the end of the file if it exists |
|`'b'`	| binary mode |
|`'t'`	| text mode (default)|
|`'+'`	| open a disk file for updating (reading and writing) |

`sys.stdin, sys.stdout, sys.error` file object는 이미 open되어 있다.

#### File methods

In [1]:
myfile = open("test.txt", "w")
myfile.write("My first file written from Python\n")
myfile.write("---------------------------------\n")
myfile.write("Hello, world!\n")
myfile.close()

In [10]:
file = open("test.txt")
file.read()

'My first file written from Python\n---------------------------------\nHello, world!\n'

In [15]:
file.read()   # returns empty string if the end of the file reached.

''

In [14]:
file.seek(0)    # go to the start of file.
file.readlines()

['My first file written from Python\n',
 '---------------------------------\n',
 'Hello, world!\n']

Old Style

In [2]:
file = open('test.txt')
for line in file.readlines():
    print(line, end='')
file.close()

My first file written from Python
---------------------------------
Hello, world!


File object is **iterable**!!
> for loop에서 한 라인씩 가져온다. 매 iteration 마다 `file.readline()`이 실행된다.
> 라인 마다 disk file을 access해야기 때문에 성능이 저하될 거라 우려할 필요없다. Buffering되어 있기 때문에 `readline()` method는 거의 memory만 access할 것이다.

In [3]:
file = open('test.txt')
for line in file:
    print(line, end='')
file.close()

My first file written from Python
---------------------------------
Hello, world!


Auto closing and exception handling

In [16]:
with open('test.txt') as f:
    for line in f:
        print(line, end='')
f.closed

My first file written from Python
---------------------------------
Hello, world!


True

## Saving structured data with JSON format
JSON(JavaScript Object Notation) is a popular data interchange format to serialize complex data types linke nested list and dict.

> 복잡한 data structure를 저장하거나 network을 통해 송신하려면, serialize해야 한다. 널리 이용되는 JSON format으로 바꿔 저장하거나 통신해라.

#### Convert Python data into json string

In [22]:
import json
data = {'pears': 217, 'apples': 430, 'oranges': 525, 'bananas': 312}
s = json.dumps(data)
s

'{"pears": 217, "apples": 430, "oranges": 525, "bananas": 312}'

#### Saving into JSON format text file

In [23]:
with open('serialized.txt', 'w') as f:
    json.dump(data, f)

#### Loading the JSON file

In [24]:
with open('serialized.txt') as f:
    inventory = json.load(f)
inventory

{'apples': 430, 'bananas': 312, 'oranges': 525, 'pears': 217}

## `pickle` module to store native Python objects

> Support persistent objects: 대부분의 Python object들을 저장할 수 있다. 하지만, 다른 언어로 작성된 application과 호환성이 없다. 

In [25]:
import pickle

# An arbitrary collection of objects supported by pickle.
data = {
    'a': [1, 2.0, 3, 4+6j],
    'b': ("character string", b"byte string"),
    'c': {None, True, False}
}

with open('data.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [27]:
with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)
data

{'a': [1, 2.0, 3, (4+6j)],
 'b': ('character string', b'byte string'),
 'c': {False, None, True}}